In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import necessary libraries and modules
from keras.applications import vgg16
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array

from keras.models import Model
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [ ]:
# image parameter

imgs_path = "/content/drive/MyDrive/tour/"
imgs_model_width, imgs_model_height = 224, 224

# number of most similar images to retrieve

no_closest_images = 5

In [ ]:
# load the model
vgg_model = vgg16.VGG16(weights='imagenet')

# remove the last layers in order to get features instead of predictions
feature_extract = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)

# print the layers of the CNN
feature_extract.summary()

In [ ]:
files = [imgs_path + x for x in os.listdir(imgs_path) if ".jpg" in x]

print("number of images:",len(files))

In [ ]:
# load an image in PIL format
original = load_img(files[1212], target_size=(imgs_model_width, imgs_model_height))
plt.imshow(original)
plt.show()

In [ ]:

numpy_image = img_to_array(original)


image_batch = np.expand_dims(numpy_image, axis=0)
print('image batch size', image_batch.shape)

# prepare the image for the VGG model
processed_image = preprocess_input(image_batch.copy())

In [ ]:
# get the extracted features
img_features = feature_extract.predict(processed_image)
print("number of image features:",img_features.size)
img_features

In [ ]:
# load all the images and prepare them for feeding into the CNN

importedImages = []

for f in files:
    filename = f
    original = load_img(filename, target_size=(224, 224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    
    importedImages.append(image_batch)
    
images = np.vstack(importedImages)

processed_imgs = preprocess_input(images.copy())


In [ ]:
# extract the images features

imgs_features = feature_extract.predict(processed_imgs)
imgs_features.shape

In [ ]:
# compute cosine similarities between images

similarity = cosine_similarity(imgs_features)

# store the results into a pandas dataframe
similarity_df = pd.DataFrame(similarity, columns=files, index=files)
similarity_df.head()

In [ ]:
# function to recommend the similar places for desired

def recommendations(given_img):

  
    print("DesiredS_place:")

    original = load_img(given_img, target_size=(imgs_model_width, imgs_model_height))
    plt.imshow(original)
    plt.show()

    
    print("--------Recommended Places:----------")

    closest_imgs = similarity_df[given_img].sort_values(ascending=False)[1:no_closest_images+1].index
    closest_imgs_scores = similarity_df[given_img].sort_values(ascending=False)[1:no_closest_images+1]

    for i in range(0,len(closest_imgs)):
        original = load_img(closest_imgs[i], target_size=(imgs_model_width, imgs_model_height))
        plt.imshow(original)
        plt.show()
        print("similarity score : ",closest_imgs_scores[i])

In [ ]:
recommendations(files[445])

In [ ]:
recommendations(files[100])

In [ ]:
recommendations(files[902])

In [ ]:
recommendations(files[1141])

In [ ]:
recommendations(files[725])

In [ ]:
recommendations(files[1078])

In [ ]:
recommendations(files[175])

In [ ]:
recommendations(files[5])